# D16 使用 Python 控制 DHT22 感測器與繼電器

# 作業1：
* 練習將 DHT22 接於 GPIO 27 接腳，並且更改軟體的接腳設定值，重新執行範例程式，驗證在不同的接腳上安裝 DHT22，程式一樣可以正確地讀出數值。

DHT22的資料線的接腳位置在GPIO 17

```
import Adafruit_DHT             # 匯入Adafruit_DHT模組
DHT_SENSOR = Adafruit_DHT.DHT22  # 感測器為DHT22
DHT_PIN = 27                    # 設定DHT22 Data接線的接腳位置
while True:
    humidity, temperature = Adafruit_DHT.read_retry(DHT_SENSOR, DHT_PIN)
    if humidity is not None and temperature is not None:
        print("Temp={0:0.1f}*C  Humidity={1:0.1f}%".format(temperature, humidity))
    else:
        print("Failed to retrieve data from humidity sensor")
```

將接腳改成17，再把DHT22的資料線也接到GPIO 17

![](https://raw.githubusercontent.com/yaws508/cupoy/main/d16/d16-01.png)

# 作業2：
* 觀察 RPi.GPIO 的程式碼，與 GPIOZero 程式碼對於繼電器控制上寫法的不同，如果我們要設定 GPIO 26, 19, 13, 6 四個接腳控制一個四路繼電器，練習實作一個 GPIOZero 四路繼電器的控制程式

#RPi.GPIO的程式碼：使用GPIO 26連接繼電器
* 5秒鐘切換一次

```
import RPi.GPIO as GPIO
from time import sleep

relay_pin = 26

GPIO.setmode(GPIO.BCM)
GPIO.setup(relay_pin, GPIO.OUT)
GPIO.output(relay_pin, 1)
try:
    while True:
        GPIO.output(relay_pin, 0)
        sleep(5)
        GPIO.output(relay_pin, 1)
        sleep(5)
except KeyboardInterrupt:
	pass
GPIO.cleanup()

```

GPIOZero 程式碼寫法:

```
#GPIOZero 程式碼：使用GPIO 26,19,13,6連接4個繼電器
import gpiozero

RELAY_PIN = 26
relay = gpiozero.OutputDevice(RELAY_PIN, active_high=False, initial_value=True)
try:
    while True:
      relay.off()
      sleep(5)
      relay.on()
      sleep(5)
except KeyboardInterrupt:
    print("\n程式結束執行\n")
    # 跳出程式，程式執行結束
    sys.exit(0)
```

* 5秒鐘切換一次

# 作業3：
* 將作業 1 與作業 2 結合，設定程式在溫度 10 度以下，打開 GPIO26，溫度 10 度以上到 20 度之間，控制 GPIO19，溫度 20 度到 30 度之間，控制 GPIO13，溫度在 30 度以上，控制 GPIO6，達成在不同的溫度區間時，控制不同的繼電器的需求。

```
import Adafruit_DHT
import sys
import time
import gpiozero


DHT_SENSOR = Adafruit_DHT.DHT22
DHT_PIN = 27
relay_pin1 = 26
relay_pin2 = 19
relay_pin3 = 13
relay_pin4 = 6
relay1 = gpiozero.OutputDevice(relay_pin1, active_high=False, initial_value=False)
relay2 = gpiozero.OutputDevice(relay_pin2, active_high=False, initial_value=False)
relay3 = gpiozero.OutputDevice(relay_pin3, active_high=False, initial_value=False)
relay4 = gpiozero.OutputDevice(relay_pin4, active_high=False, initial_value=False)

def set_relay(status): #定義繼電器開關
    if status:
        print("relay on")
        relay1.on()
        relay2.on()
        relay3.on()
        relay4.on()

    else:
        print("relay off")
        relay1.off()
        relay2.off()
        relay3.off()
        relay4.off()

def main_loop():
    while True:
        humidity, temperature = Adafruit_DHT.read_retry(DHT_SENSOR, DHT_PIN)
        if humidity is not None and temperature is not None:
            print("Temp={0:0.1f}*C Humidity={1:0.1f}%".format(temperature, humidity))
            if temperature <= 10:
                relay1.on()
                relay2.off()
                relay3.off()
                relay4.off()
                print("relay1 is  on")  
            if temperature > 10 and  temperature <= 20:
                relay1.off()
                relay2.on()
                relay3.off()
                relay4.off()
                print("relay2 is  on") 
            if temperature > 20 and  temperature <= 30:
                relay1.off()
                relay2.off()
                relay3.on()
                relay4.off()
                print("relay3 is  on")
            if temperature > 30:
                relay1.off()
                relay2.off()
                relay3.off()
                relay4.on()
                print("relay4 is  on") 
            time.sleep(1)
        else:
            print("Failed to retrieve data from humidity sensor")

if __name__=="__main__":
    try:
        main_loop()
    except KeyboardInterrupt:
        set_relay(False) #關閉程式時可將所有繼電器關閉
        print("Exit\n")
        sys.exit(0)
```

![](https://raw.githubusercontent.com/yaws508/cupoy/main/d16/d16-02.png)